In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


In [2]:
df = pd.read_excel('HUN Delivery Data Hackathon_YTD Aug25.xlsx')
display(df)

,Calendar Day,Month,CCR Delivery Type,Delivery Number,Equipment Number,Plant,S2 Start Load Time,S6 Start Unload TS,ShipTo Number,Sold-To Number,Delivered Volume (SUM)
0,20250603,6,DE,HLDU25-00001,LHV204,HU3I,95200,0000/00/00 11:00:00,18332139.0,18200651,7.5
1,20250603,6,DE,HLDU25-00002,AEMJ879,HU3I,100700,0000/00/00 10:45:00,18332139.0,18200651,6.0
2,20250603,6,CO,HLDU25-00003,NYW495,HU3I,102151,0000/00/00 10:39:51,18333969.0,18209586,12.0
3,20250603,6,DE,HLDU25-00004,LEN036,HU3I,104300,0000/00/00 11:20:00,18332139.0,18200651,6.0
4,20250603,6,CO,HLDU25-00005,MZP099,HU3I,111500,0000/00/00 11:18:13,18286387.0,18200044,1.5
...,...,...,...,...,...,...,...,...,...,...,...
3636,20250829,8,CO,SLDU25-03640,PSD862,HU3I,150006,0000/00/00 15:09:06,18334481.0,18210979,5.0
3637,20250829,8,DE,SLDU25-03641,SCG036,HU3I,152300,0000/00/00 15:55:00,18333205.0,18201732,2.5
3638,20250829,8,DE,SLDU25-03642,AEMJ879,HU3I,155100,0000/00/00 16:20:00,18332139.0,18200651,4.0
3639,20250830,8,DE,SLDU25-03643,SJC752,HU3I,103500,0000/00/00 11:20:00,18332139.0,18200651,6.0


In [3]:
subset = df[(df["Calendar Day"] == 20250603) & (df["CCR Delivery Type"] == "DE")]

summi = 0
for index, row in subset.iterrows():
    rest = row["Delivered Volume (SUM)"] % 10
    summi += abs(rest - 10)
    
print(summi)


118.5


In [4]:
# Waste berechnen
df["waste"] = ((df["Delivered Volume (SUM)"] % 10) - 10).abs()

# Nur DE-Lieferungen berücksichtigen
df_de = df[df["CCR Delivery Type"] == "DE"].copy()

# Gruppierung pro Tag
result = df_de.groupby("Calendar Day").agg(
    orders=("Delivered Volume (SUM)", "count"),
    total_waste=("waste", "sum"),
    total_volume=("Delivered Volume (SUM)", "sum")
)

print(result)
arr = []
arr.append( result["total_waste"].sum())
arr.append(result["total_volume"].sum())
labels = ["Total Waste", "Total Volume"]
print(result["orders"].mean())


              orders  total_waste  total_volume
Calendar Day                                   
20250106           1         9.00          1.00
20250107           2         4.76         15.24
20250108           1         1.90          8.10
20250109           2         4.76         15.24
20250113           2         7.00         13.00
...              ...          ...           ...
20250826           5        14.00         36.00
20250827           3         9.60         20.40
20250828           2         5.00         15.00
20250829          23        94.00        176.00
20250830           1         4.00          6.00

[176 rows x 3 columns]
16.920454545454547


In [5]:
# Gesamtsummen
total_waste = result["total_waste"].sum()
total_volume = result["total_volume"].sum()

arr = [total_waste, total_volume]
labels = ["Total Waste", "Total Volume"]


fig_percent = px.pie(
    names=labels,
    values=arr,
    color=labels,
    color_discrete_sequence=px.colors.sequential.Reds,
    title="Waste vs. Delivered Concrete (% und m³)"
)

fig_percent.update_traces(
    textinfo="percent+value+label",   # ⬅ Wichtig: zeigt % + m³ + Label
    pull=[0.05, 0],
    hovertemplate="<b>%{label}</b><br>%{percent}<br>%{value:.2f} m³"
)

fig_percent.update_layout(showlegend=False)

fig_percent.write_html("percent_plot.html")


In [6]:
df[df["Calendar Day"]==20250603]["Delivered Volume (SUM)"].sum()

324.25

In [14]:
df2 = pd.read_excel('HUN Sales Database_YTD Aug25.xlsx')

df2[df2["Calendar Day"]==20250603]["Delivered Volume (SUM)"].sum()
display(df2)

KeyError: 'Calendar Day'

In [8]:

df2 = df2.rename(columns={"Delivery note": "Delivery Number"})
df_all = pd.merge(df, df2, on=["Delivery Number"])

0    2424
1    2400
2    2400
3    2440
4    2400
Name: Postal code, dtype: object

In [15]:
df_single_day = df_all[(df_all["CCR Delivery Type"] == "DE") & (df_all["Calendar Day"]==20250603)]
display(df_single_day.groupby(["Strength","Dmax","Consistency"]).count())

df_single_day = df_single_day.rename(columns={"Delivered Volume (SUM)": "Volume"})
df_single_day[["Calendar Day","Strength","Dmax","Consistency","Volume","Postal code"]].to_csv("single_day.csv")

Calendar Day  Month_x  CCR Delivery Type  \
Strength Dmax Consistency                                             
25/30    16.0 F3                      1        1                  1   
         24.0 F3                      4        4                  4   
30/37    8.0  F4                      1        1                  1   
         16.0 F1                      1        1                  1   
              F3                     18       18                 18   
              F4                      1        1                  1   
         24.0 F3                      2        2                  2   
              F4                     10       10                 10   

                           Delivery Number  Equipment Number  Plant_x  \
Strength Dmax Consistency                                               
25/30    16.0 F3                         1                 1        1   
         24.0 F3                         4                 4        4   
30/37    8.0  F4                         1                 1        1   
         16.0 F1                         1                 1        1   
              F3                        18                18       18   
              F4                         1                 1        1   
         24.0 F3                         2                 2        2   
              F4                        10                10       10   

                           S2 Start Load Time  S6 Start Unload TS  \
Strength Dmax Consistency                                           
25/30    16.0 F3                            1                   1   
         24.0 F3                            4                   4   
30/37    8.0  F4                            1                   1   
         16.0 F1                            1                   1   
              F3                           18                  18   
              F4                            1                   1   
         24.0 F3                            2                   2   
              F4                           10                  10   

                           ShipTo Number  Sold-To Number  ...  Month_y  \
Strength Dmax Consistency                                 ...            
25/30    16.0 F3                       1               1  ...        1   
         24.0 F3                       4               4  ...        4   
30/37    8.0  F4                       1               1  ...        1   
         16.0 F1                       1               1  ...        1   
              F3                      18              18  ...       18   
              F4                       1               1  ...        1   
         24.0 F3                       2               2  ...        2   
              F4                      10              10  ...       10   

                           Location  Postal code  ASP m3  Cost m3  MoM m3  \
Strength Dmax Consistency                                                   
25/30    16.0 F3                  1            1       1        1       1   
         24.0 F3                  4            4       4        4       4   
30/37    8.0  F4                  1            1       1        1       1   
         16.0 F1                  1            1       1        1       1   
              F3                 18           18      18       18      18   
              F4                  1            1       1        1       1   
         24.0 F3                  2            2       2        2       2   
              F4                 10           10      10       10      10   

                           Revenue  Cost  MoM  Band  
Strength Dmax Consistency                            
25/30    16.0 F3                 1     1    1     1  
         24.0 F3                 4     4    4     4  
30/37    8.0  F4                 1     1    1     1  
         16.0 F1                 1     1    1     1  
              F3                18    18   18    18  
    

In [19]:
df_single_day["Postal code"].value_counts()


Postal code
Hank    37
Iván     1
Name: count, dtype: int64

In [11]:

# Funktion berechnet direkt den "Luft"-Anteil im letzten Truck
def calc_unused_space(vol):
    rest = vol % 12  # Was passt nicht in die vollen 12er?
    
    if rest == 0: return 0        # Geht genau auf
    if rest <= 3: return 3 - rest # Rest kommt in 3er -> Differenz ist Luft
    if rest <= 7: return 7 - rest # Rest kommt in 7er -> Differenz ist Luft
    return 12 - rest              # Rest muss in 12er -> Differenz ist Luft

# 1. Berechnung für jede Zeile anwenden
df_single_day["Unused_Space"] = df_single_day["Volume"].apply(calc_unused_space)

# 2. Gesamtsumme der transportierten Luft ausgeben
total_unused = df_single_day["Unused_Space"].sum()
total_volume = df_single_day["Volume"].sum()

print(f"Gesamtes transportiertes Volumen: {total_volume} m³")
print(f"Gesamtes Leervolumen (Luft):     {total_unused} m³")


Gesamtes transportiertes Volumen: 261.5 m³
Gesamtes Leervolumen (Luft):     121.5 m³


In [12]:
import pandas as pd
import plotly.graph_objects as go

# Hilfsspalte für den Rest berechnen (Was passt nicht in volle 12er Ladungen?)
df_single_day['Remainder'] = df_single_day['Volume'] % 12

# 1. Anzahl 12m³ Trucks:
# Das sind alle vollen Ladungen (Ganzzahl-Division //)
# PLUS ein weiterer 12er, falls der Rest > 7m³ ist (da er nicht in den 7er passt)
df_single_day['Count_12'] = (df_single_day['Volume'] // 12) + (df_single_day['Remainder'] > 7).astype(int)

# 2. Anzahl 7m³ Trucks:
# Nur wenn der Rest zwischen 3 und 7 liegt
df_single_day['Count_7'] = ((df_single_day['Remainder'] > 3) & (df_single_day['Remainder'] <= 7)).astype(int)

# 3. Anzahl 3m³ Trucks:
# Nur wenn der Rest zwischen 0 und 3 liegt
df_single_day['Count_3'] = ((df_single_day['Remainder'] > 0) & (df_single_day['Remainder'] <= 3)).astype(int)

# --- Summen bilden ---
total_12 = df_single_day['Count_12'].sum()
total_7  = df_single_day['Count_7'].sum()
total_3  = df_single_day['Count_3'].sum()

print(f"Total 12m³ Trucks: {total_12}")
print(f"Total 7m³  Trucks: {total_7}")
print(f"Total 3m³  Trucks: {total_3}")

# --- Visualisierung (Bar Chart) ---
# Ein Bar Chart ist hier besser als ein Pie Chart, um die absolute Anzahl zu vergleichen

fig = go.Figure([go.Bar(
    x=['3m³ Truck', '7m³ Truck', '12m³ Truck'],
    y=[total_3, total_7, total_12],
    text=[total_3, total_7, total_12],
    textposition='auto',
    marker_color=['#E6E6DF', '#004E2B', '#00DD39'] # Hellgrün zu Dunkelgrün
)])

fig.update_layout(
    title_text='Total Truck Count by Type',
    yaxis_title='Number of Trucks',
    xaxis_title='Truck Type',
    template='plotly_white'
)

fig.show()

Total 12m³ Trucks: 25.0
Total 7m³  Trucks: 11
Total 3m³  Trucks: 2


In [13]:
import plotly.graph_objects as go

colors = ['#00DD39', '#004E2B']  # Grün für Volume, Hellgrau für Air

fig = go.Figure(data=[go.Pie(
    labels=['Delivered Volume', 'Unused Space (Air)'],
    values=[total_volume, total_unused],

    textinfo='value+percent',
    texttemplate='%{value:.1f} m³<br><b>%{percent}</b>',

    # Hervorhebung: Segment ein wenig herausziehen (Pull-Out)
    pull=[0, 0.08],   # 2. Slice etwas rausgezogen

    # Schatten-Imitation durch Rand
    marker=dict(
        colors=colors,
        line=dict(color='black', width=2),
        pattern=dict(
            shape=["", "/"],   # leichter Pattern für „Air“ als Textur
            solidity=0.9
        )
    ),

    textfont=dict(size=18, color='white'),
    hoverinfo='label+value+percent',
)])

fig.update_layout(
    title_text="2025-06-03 Truck Efficiency: Payload vs. Empty Space",
    title_font_size=24,
    showlegend=True,

    # Leichter 3D-Effekt durch Rotation und Winkel
    piecolorway=colors
)

fig.show()
